# Dashboard Development - using Dash/JupyterDash

The `jupyter-dash` package makes it easy to develop Plotly Dash apps from the Jupyter Notebook and JupyterLab.  
Just replace the standard `dash.Dash` class with the `jupyter_dash.JupyterDash` subclass.

When running in JupyterHub (or Binder), call the `infer_jupyter_config` function to detect the proxy configuration. This will detect the proper request_pathname_prefix and server_url values to use when displaying Dash apps.  
For example:  
- server_url = `https://jupyter-jsc.fz-juelich.de`  
- request_pathname_prefix = `/user/j.goebbert@fz-juelich.de/jureca_login/`  

For details please check the source here https://github.com/plotly/jupyter-dash/blob/v0.2.1.post1/jupyter_dash/comms.py#L33

In [ ]:
from jupyter_dash import JupyterDash
# https://github.com/plotly/jupyter-dash/blob/v0.3.0/jupyter_dash/jupyter_app.py#L45
JupyterDash.infer_jupyter_proxy_config()

**Attention:** I have to run this cell twice: first press play, wait a bit and hit play again while it still shows `[*]`

**Bug:** this seems to run all `*.py` files in the current folder.  
I do not know why and how that happens, but it should be fixed.

-------------------------
## Preparation

### before use in JupyterLab
- create directory 'logs'
- create directory 'cache'
- create link to 'assets'  

in app.py
- dash_app = JupyterDash(..)
- dash_app = app
- set cache_dir = "./cache"
- set cache_time = 86400

In [ ]:
!mkdir -p logs
!mkdir -p cache

### Tests data

In [ ]:
## load & initialize static data
# => tuple-list 'counties_geojson'
# => dataframe 'counties_metadf'
import json
import pandas as pd

# Landkreis-Geometrie vom RKI:
# https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0 -> ShapeFile
# https://mapshaper.org -> GEOJSON

with open('assets/DE-Landkreise_RKI.geojson.json') as json_file:
    counties_geojson = json.load(json_file)

# 'id' must be the unique identifier of the shapes (required by plotly.Choroplethmapbox)
for i, feat in enumerate(counties_geojson['features']):
    feat['id'] = i

counties_metadf = pd.DataFrame(data={
        # geoid must match county-shape-ids in json file (must start with 0 and increase with no missing one)
        'geoids' :    [feat['id']                             for feat in counties_geojson['features']],
        'names'  :    [feat['properties'].get('GEN')          for feat in counties_geojson['features']],
        'cca'    :    [feat['properties'].get('RS')           for feat in counties_geojson['features']],
    })

# test 
counties_metadf.loc[counties_metadf['cca']=='03159'] # data of Göttingen
#counties_metadf.to_csv('counties_metadata.csv', index=False)

------------------------------
## Run the Dashboard

### load all with `import app.py`



In [ ]:
import os

port = 8058
base_url = "https://jupyter-jsc.fz-juelich.de"
prefix_path = os.getenv('JUPYTERHUB_SERVICE_PREFIX') + "proxy/{}".format(port) + "/"
print(base_url + prefix_path)

os.environ["BASE_URL"] = base_url
os.environ["PREFIX_PATH"] = prefix_path

os.environ["MIN_DATE"] = "2020_08_01"
os.environ["MAX_DATE"] = "2020_11_30"
os.environ["INIT_DATE"] = "2020_11_01"

# switch from PNGs to CSVs (first date with CSVs)
os.environ["THRESHOLD_DATE"] = "2020_10_19"

In [ ]:
#%reload_ext autoreload
#%autoreload 2

# ensure we create `app` in app.py with JupyterDash
from plotly_figures.maps import *
import app

### more tests

In [ ]:
# test init dir location
from datetime import datetime as dt
# test_date = app.init_date
test_date = dt.strptime("2020_10_18", '%Y_%m_%d').date() 
assets_dir = app.get_assets_datadir(dt(test_date.year, test_date.month, test_date.day))
assets_dir

In [ ]:
# test plot function
init_mapcsv_path = "{}map.csv".format(app.init_assets_path)
print(init_mapcsv_path)

init_mapfig_bstim = create_map_figure(
    app.counties_geojson,
    app.counties_metadf,
    init_mapcsv_path,
    column='newInf100k',
)

init_mapfig_bstim.show(
    config={"displayModeBar": False,
            "showTips": False,
            "modeBarButtonsToRemove": ['toImage'],
    }
) # "staticPlot": True})

### start the dashboard

In [ ]:
print("Dash app running on", base_url + prefix_path)
app.dash_app.run_server(mode="external", debug=True, port=port)
    # mode="jupyterlab" -> will open the app in a tab in JupyterLab
    # mode="inline"     -> will open the app below this cell
    # mode="external"   -> will displays a URL that you can MIGHT click on to open the app in a browser tab
    #                   -> it it is 127.0.0.1 then take `print(base_url + prefix_path)`